In [1]:
!apt install mongodb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libpcap0.8 libstemmer0d libyaml-cpp0.5v5 mongo-tools mongodb-clients
  mongodb-server mongodb-server-core
The following NEW packages will be installed:
  libpcap0.8 libstemmer0d libyaml-cpp0.5v5 mongo-tools mongodb mongodb-clients
  mongodb-server mongodb-server-core
0 upgraded, 8 newly installed, 0 to remove and 21 not upgraded.
Need to get 53.1 MB of archives.
After this operation, 215 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpcap0.8 amd64 1.8.1-6ubuntu1.18.04.2 [118 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libstemmer0d amd64 0+svn585-1build1 [62.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 lib

In [2]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [3]:
!pip install pymongo


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from pymongo import MongoClient
from bs4 import BeautifulSoup as bs
import requests
from time import  sleep
import json

In [34]:
url_hh = 'https://novosibirsk.hh.ru/search/vacancy?text=python&from=suggest_post&area=4&hhtmFrom=vacancy_search_list'
url_rabota='https://nsk.rabota.ru/vacancy/?query=python&sort=relevance'

headers={
    'User-Agent':	'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
}
def hh_parsing():
  vacancy_list_hh=[]
  for page in range(200):
    sleep(3) 
    url=url_hh+f'&page={page}'
    response_hh=requests.get(url, headers=headers)
    dom_hh = bs(response_hh.content, 'html.parser')
    vacancies=dom_hh.find_all('div', {'class': 'vacancy-serp-item-body__main-info'})

    vacancy_list=[]
    for vacancy in vacancies:  
      vacancy_name=vacancy.find('a', {'class': 'serp-item__title'}).text
      vacancy_link=vacancy.find('a', {'class': 'serp-item__title'})['href']
      vacancy_salary=vacancy.find('span', {'class': 'bloko-header-section-3'})
      vacancy_employee=vacancy.find('a', {'class': 'bloko-link bloko-link_kind-tertiary'}).text.replace('\xa0', '')
      
      if vacancy_salary:
        vacancy_salary=vacancy_salary.text.replace('\u202f', '').split(' ')       
        for elements in vacancy_salary:
          if len(vacancy_salary)==4:
            salary_min=vacancy_salary[0]
            salary_max=vacancy_salary[2]
            salary_currency=vacancy_salary[-1]
          elif len(vacancy_salary)==3:
            salary_min=vacancy_salary[1]
            salary_max='по договоренности'
            salary_currency=vacancy_salary[-1]
      else:
        salary_min, salary_max, salary_currency=None, None, None


      vacancy_dict={
          'vacancy_name': vacancy_name,
          'vacancy_link': vacancy_link,
          'salary_min': salary_min,
          'salary_max': salary_max,
          'salary_currency': salary_currency,
          'vacancy_employee': vacancy_employee,
          'resource': 'novosibirsk.hh.ru',
    
      }
      vacancy_list.append(vacancy_dict)
    
    if len(vacancy_list)>1:
      vacancy_list_hh.append(vacancy_list)
    else:
      return vacancy_list_hh
    

just_parsed_vacancies_hh=hh_parsing()


In [35]:
just_parsed_vacancies_hh

[[{'vacancy_name': 'Разработчик ПО - Backend (Python)',
   'vacancy_link': 'https://novosibirsk.hh.ru/vacancy/75748706?from=vacancy_search_list&query=python',
   'salary_min': '150000',
   'salary_max': 'по договоренности',
   'salary_currency': 'руб.',
   'vacancy_employee': 'АОСмартком',
   'resource': 'novosibirsk.hh.ru'},
  {'vacancy_name': 'Data Engineer (Big Data)',
   'vacancy_link': 'https://novosibirsk.hh.ru/vacancy/75522708?from=vacancy_search_list&query=python',
   'salary_min': '180000',
   'salary_max': '220000',
   'salary_currency': 'руб.',
   'vacancy_employee': 'Астор',
   'resource': 'novosibirsk.hh.ru'},
  {'vacancy_name': 'Разработчик Python (R&D)',
   'vacancy_link': 'https://novosibirsk.hh.ru/vacancy/74176205?from=vacancy_search_list&query=python',
   'salary_min': None,
   'salary_max': None,
   'salary_currency': None,
   'vacancy_employee': 'ROBOTMIA',
   'resource': 'novosibirsk.hh.ru'},
  {'vacancy_name': 'Программист Python',
   'vacancy_link': 'https://novo

In [36]:
def rabota_ru_parsing():
  
  response_rabota=requests.get(url_rabota, headers=headers)
  dom_rabota = bs(response_rabota.content, 'html.parser')
  vacancies=dom_rabota.find_all('div', {'class': 'vacancy-preview-card__top'})

  vacancy_list_rabota_ru=[]
  for vacancy in vacancies:
    vacancy_list=[]
    sleep(3)
    vacancy_name=vacancy.find('h3', {'class': 'vacancy-preview-card__title'}).text
    vacancy_link='https://nsk.rabota.ru/'+vacancy.find('a', {'class': 'vacancy-preview-card__title_border'})['href']
    vacancy_salary=vacancy.parent.find('div', {'class': 'vacancy-preview-card__salary'}).text
    vacancy_employee=vacancy.find('span', {'class':'vacancy-preview-card__company-name'}).text
    

    if vacancy_salary:
      vacancy_salary=vacancy_salary.split(' ')
        
      for elements in vacancy_salary:
        if vacancy_salary[0]=='до':
          salary_min='не утсановлено'
          salary_max=vacancy_salary[1]
          salary_currency=vacancy_salary[-1]
        elif vacancy_salary[0]=='от':
          salary_min=vacancy_salary[1]
          salary_max='не утсановлено'
          salary_currency=vacancy_salary[-1]
        elif vacancy_salary[0]=='по':
          tmp= vacancy_salary[0]+' '+ vacancy_salary[1]
          salary_min, salary_max, salary_currency=tmp, tmp, tmp
        else:
          salary_min=vacancy_salary[0]
          salary_max=vacancy_salary[1]
          salary_currency=vacancy_salary[-1]

          vacancy_dict={
            'vacancy_name': vacancy_name,
            'vacancy_link': vacancy_link,
            'salary_min': salary_min,
            'salary_max': salary_max,
            'salary_currency': salary_currency,
            'vacancy_employee': vacancy_employee,
            'resource': 'rabota_ru',
      
        }
          
          
          vacancy_list.append(vacancy_dict)
      
    if len(vacancy_list)>1:
        vacancy_list_rabota_ru.append(vacancy_list)
    else:
      with open('vacancy_list_rabota_ru.json', 'w') as json_rabota_ru_parsing:
        json.dump(vacancy_list_rabota_ru, json_rabota_ru_parsing)
        
  return vacancy_list_rabota_ru

just_parsed_vacancies_rabota_ru=rabota_ru_parsing()


In [37]:
just_parsed_vacancies_rabota_ru

[[{'vacancy_name': '\n              Системный администратор\n            ',
   'vacancy_link': 'https://nsk.rabota.ru//vacancy/47127997/?search_id=1673545254042vkr3c3vudcl',
   'salary_min': '80\xa0000\xa0—',
   'salary_max': '200\xa0000',
   'salary_currency': 'руб.',
   'vacancy_employee': '\n              Jobers\n            ',
   'resource': 'rabota_ru'},
  {'vacancy_name': '\n              Системный администратор\n            ',
   'vacancy_link': 'https://nsk.rabota.ru//vacancy/47127997/?search_id=1673545254042vkr3c3vudcl',
   'salary_min': '80\xa0000\xa0—',
   'salary_max': '200\xa0000',
   'salary_currency': 'руб.',
   'vacancy_employee': '\n              Jobers\n            ',
   'resource': 'rabota_ru'},
  {'vacancy_name': '\n              Системный администратор\n            ',
   'vacancy_link': 'https://nsk.rabota.ru//vacancy/47127997/?search_id=1673545254042vkr3c3vudcl',
   'salary_min': '80\xa0000\xa0—',
   'salary_max': '200\xa0000',
   'salary_currency': 'руб.',
   'va

In [38]:
def number_of_just_parsed_vacansies(lists_of_just_parsed_vacancies):
  
  number_of_just_parsed_vacansies=0
  for lists in lists_of_just_parsed_vacancies:
    for vacancies in lists:
      number_of_just_parsed_vacansies+=1
  return number_of_just_parsed_vacansies

number_of_just_parsed_vacansies(just_parsed_vacancies_hh), number_of_just_parsed_vacansies(just_parsed_vacancies_rabota_ru)  

(100, 33)

In [40]:
vacancy_list_hh='/content/vacancy_list_hh.json'
vacancy_list_rabota_ru='/content/vacancy_list_rabota_ru.json'

with open(vacancy_list_hh) as file_hh:
    vacancy_hh = json.load(file_hh)

with open(vacancy_list_rabota_ru) as file_rabota_ru:
    vacancy_rabota_ru = json.load(file_rabota_ru)

client=MongoClient()
db_hh=client.vacancy_hh
db_rabota_ru=client.vacancy_rabota_ru

for listst_from_vacancy_hh in vacancy_hh:
  db_hh.vacancy_hh.insert_many(listst_from_vacancy_hh)

for listst_from_vacancy_rabota_ru in vacancy_rabota_ru:
  db_hh.vacancy_rabota_ru.insert_many(listst_from_vacancy_rabota_ru)

downloaded_number_of_vacancies_hh=len(list(db_hh.vacancy_hh.find()))
downloaded_number_of_vacancies_rabota_ru=len(list(db_hh.vacancy_rabota_ru.find()))
downloaded_number_of_vacancies_hh, downloaded_number_of_vacancies_rabota_ru

(200, 42)

In [46]:
def check_and_update_db_hh(list_vacancies_need_to_chek_and_add_to_db):
  for list_vacancies in list_vacancies_need_to_chek_and_add_to_db:
    for vacancies in list_vacancies:
      if not db_hh.vacancy_hh.find({'vacancy_link': vacancies['vacancy_link'], 'vacancy_employee': vacancies['vacancy_employee']}):
        db_hh.vacancy_hh.insert_one(vacancies)

def check_and_update_db_rabota_ru(list_vacancies_need_to_chek_and_add_to_db):
  for list_vacancies in list_vacancies_need_to_chek_and_add_to_db:
    for vacancies in list_vacancies:
      if not db_rabota_ru.vacancy_rabota_ru.find({'vacancy_link': vacancies['vacancy_link'], 'vacancy_employee': vacancies['vacancy_employee']}):
        db_rabota_ru.vacancy_rabota_ru.insert_one(vacancies)      

check_and_update_db_hh(just_parsed_vacancies_hh)
check_and_update_db_rabota_ru(just_parsed_vacancies_rabota_ru)

In [47]:
def how_much_vacancies_added_into_db_hh():
  return len(list(db_hh.vacancy_hh.find()))-downloaded_number_of_vacancies_hh

In [48]:
def how_much_vacancies_added_into_db_rabota_ru():
  return len(list(db_hh.vacancy_rabota_ru.find()))-downloaded_number_of_vacancies_rabota_ru

In [49]:
print(f'There was added {how_much_vacancies_added_into_db_hh()} new vacancies')
print(f'There was added {how_much_vacancies_added_into_db_rabota_ru()} new vacancies')

There was added 0 new vacancies
There was added 0 new vacancies
